<a href="https://colab.research.google.com/github/pschrader98/ACC540_Tutorial5/blob/main/ACC540Tutorial5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import torch

In [2]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

In [3]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone', model_max_length=512)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [4]:
data = pd.read_csv('https://drive.google.com/file/d/1GH10GyeLEpaT-aeDdugZ1juZNiTY4twg/view?usp=sharing')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 3, saw 1287


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
finbert.to(device)

In [ ]:
predictions = []
for out in nlp(df_comments_ticker['Body'].tolist())):
    predictions.append(out)

predictions